In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import metrics
from sklearn import ensemble

import datetime
import time

2022-05-27 10:45:03.903184: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/gurobi951/linux64/lib
2022-05-27 10:45:03.903256: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
data_train = pd.read_csv("data/css_main_training.csv", sep=";")
data_test = pd.read_csv("data/css_main_test.csv", sep=";")
data_fuel = pd.read_csv("data/fuel_prices.csv", sep=";")

In [5]:
to_drop = ["id_contract", "id_payer", "temperature", "route_start_datetime", "route_end_datetime", "route_start_unix", "route_end_unix", "date"]
ordinal_cats = ["load_size_type", "contract_type"]
one_hot_cats = ["direction", "route_start_country", "last_unload_country", "first_load_country", "route_end_country", "id_currency", "prim_train_line",
                "prim_ferry_line", "id_service_type"]


In [6]:
def preproc(df, train=True, enc_one_hot=None, enc_ordinal=None):
    if train:
        assert(enc_one_hot is None and enc_ordinal is None)
    else:
        assert(enc_one_hot is not None and enc_ordinal is not None)

    df = df.copy()

    nan_to_zero = ["prim_train_line", "prim_ferry_line"]
    for col_name in nan_to_zero:
        df[col_name] = df.apply(lambda row: 0 if np.isnan(row[col_name]) else row[col_name], axis=1)

    df["route_start_unix"] = df.apply(lambda row: time.mktime(datetime.datetime.fromisoformat(row["route_start_datetime"]).timetuple()), axis=1)
    df["route_end_unix"] = df.apply(lambda row: time.mktime(datetime.datetime.fromisoformat(row["route_end_datetime"]).timetuple()), axis=1)
    df["duration"] = df.apply(lambda row: row["route_end_unix"] - row["route_start_unix"], axis=1)
    df["date"] = df.apply(lambda row: row["route_start_datetime"].split()[0], axis=1)

    df = df.merge(data_fuel, on="date")

    df.drop(columns=to_drop, inplace=True)

    if train:
        df.dropna(inplace=True)

    if train:
        df = df.sample(frac=1, random_state=18)

    X, y = df.drop(columns=["expenses"] + one_hot_cats + ordinal_cats).to_numpy(), df["expenses"].to_numpy()

    df_one_hot_cats = df[one_hot_cats]
    if train:
        enc_one_hot = preprocessing.OneHotEncoder(handle_unknown="ignore")
        X_one_hot_cats = enc_one_hot.fit_transform(df_one_hot_cats.to_numpy()).toarray()
    else:
        X_one_hot_cats = enc_one_hot.transform(df_one_hot_cats.to_numpy()).toarray()
    X = np.concatenate((X, X_one_hot_cats), axis=1)

    df_ordinal_cats = df[ordinal_cats]
    if train:
        enc_ordinal = preprocessing.OrdinalEncoder()
        X_ordinal_cats = enc_ordinal.fit_transform(df_ordinal_cats.to_numpy())
    else:
        X_ordinal_cats = enc_ordinal.transform(df_ordinal_cats.to_numpy())
    X = np.concatenate((X, X_ordinal_cats), axis=1)


    print(X.shape)
    if train:
        return X, y, enc_one_hot, enc_ordinal
    return X, y

In [ ]:
def validate(model, X, y, test_size=0.2):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = metrics.mean_squared_error(y_test, y_pred) ** (1/2)
    return -score

In [ ]:
def make_submit(model):
    X_train, y_train, enc_one_hot, enc_ordinal = preproc(data_train, train=True)
    X_test, _ = preproc(data_test, train=False, enc_one_hot=enc_one_hot, enc_ordinal=enc_ordinal)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    assert(len(y_pred) == len(data_test.index))
    print(y_pred)
    pd.Series(y_pred).to_csv(f"result.txt", sep='\n', header=False, index=False)

In [ ]:
X_train, y_train, enc_one_hot, enc_ordinal = preproc(data_train, train=True)
X_test, y_test = preproc(data_test, train=False, enc_one_hot=enc_one_hot, enc_ordinal=enc_ordinal)

In [ ]:
model = #TODO!!!

In [ ]:
validate(model, X_train, y_train)

In [ ]:
make_submit(model)